In [ ]:
# Add the relevant scripts from LArMachineLearningData
# Nice the process so it can run with lots of cores on low priority
import os
os.nice(20)

# Add local paths
import sys
hnlDIR = os.environ['_']
sys.path.append('../pyscript')

# From pyscript Library
from Plotting import *
from Dictionary import *
from HelperFunctions import *


import uproot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
import random

from ROOT import TLorentzVector
from ROOT import TVector3

from pylorentz import Momentum4

import warnings
warnings.filterwarnings("ignore")

In [ ]:
c_cm_per_ns = 29.9792458

SBND_AV = [{
      "xmin": -200,
      "xmax": 0,
      "ymin": -200,
      "ymax": 200,  
      "zmin": 0,
      "zmax": 500,
    },
    {
      "xmin": 0,
      "xmax": 200,
      "ymin": -200,
      "ymax": 200,
      "zmin": 0,      
      "zmax": 500,
    }
]

<h3> Read nu DF</h3>

In [ ]:
def read_nuroot2df(fname):
    #SPECIFY BRANCHES
    branches = [
        "eventID"
        , "vtxX"
        , "vtxY"
        , "vtxZ"
        , "vtxT"
    ]
    

    folder = "ana"
    tname = "tree"
    tree = uproot.open(fname)[folder][tname]
    df = tree.arrays(branches, library="pd")
    return df

def prep_nudf(df):
    
    #APPLY AV cut: only keep vtx inside TPc
    df= df.loc[(df['vtxX'] > SBND_AV[0]['xmin'])
             & (df['vtxX'] < SBND_AV[1]['xmax'])
             & (df['vtxY'] > SBND_AV[0]['ymin'])
             & (df['vtxY'] < SBND_AV[1]['ymax'])
             & (df['vtxZ'] > SBND_AV[0]['zmin'])
             & (df['vtxZ'] < SBND_AV[1]['zmax'])]

    return df

def correct_nuZ(df):
    df['nu_enterT'] = df['vtxT'] - df['vtxZ'] / c_cm_per_ns
    return df
    

nuName = "nu_truth.root"
dfNU = read_nuroot2df(nuName)
dfNU = prep_nudf(dfNU)
dfNU = correct_nuZ(dfNU)


<h3> Read HNL DF</h3>

In [ ]:
def read_hnlroot2df(fname):
    #SPECIFY BRANCHES
    branches = [
    "mevprtl_mom"
    , "meson_dmom"
    , "decay_pos"
    , "daughter_mom"
    , "daughter_e"
    , "daughter_pdg"
    , "pot"
    , "flux_weight"
    , "ray_weight"
    , "decay_weight"

    ]
    
    folder = "generator"
    tname = "mevprtl_gen"
    tree = uproot.open(fname)[folder][tname]
    df = tree.arrays(branches, library="pd")
    return df

def prep_hnldf(df):
    #FLATTEN MULTI INDEX COLUMN FOR EASY HANDLING
    df.columns = ['_'.join(col) for col in df.columns.values]

    #RENAME COLUMN FOR CLARITY
    df = df.rename(columns = {
    "mevprtl_mom_fP_fX" : "hnl_mom_pX"
    , "mevprtl_mom_fP_fY" : "hnl_mom_pY"
    , "mevprtl_mom_fP_fZ" : "hnl_mom_pZ"
    , "mevprtl_mom_fE_" : "hnl_mom_E"
    , "meson_dmom_fP_fX" : "kaon_mom_pX"
    , "meson_dmom_fP_fY" : "kaon_mom_pY"
    , "meson_dmom_fP_fZ" : "kaon_mom_pZ"
    , "meson_dmom_fE_" : "kaon_mom_E"
    , "decay_pos_fP_fX" : "hnl_decay_X"
    , "decay_pos_fP_fY" : "hnl_decay_Y"
    , "decay_pos_fP_fZ" : "hnl_decay_Z"
    , "decay_pos_fE_" : "hnl_decay_t"
    , "daughter_momfX__" : "daughter_mom_pX"
    , "daughter_momfY__" : "daughter_mom_pY"
    , "daughter_momfZ__" : "daughter_mom_pZ"
    , "daughter_e__" : "daughter_mom_E"
    , "daughter_pdg__" : "daughter_pdg"    
    , "pot__" : "pot"
    , "flux_weight__" : "fluxw"
    , "ray_weight__" : "rayw"
    , "decay_weight__" : "decayw"
    }
    )
    
    #APPLY AV cut the HNL DECAY VERTEX
    df= df.loc[(df['hnl_decay_X'] > SBND_AV[0]['xmin'])
             & (df['hnl_decay_X'] < SBND_AV[1]['xmax'])
             & (df['hnl_decay_Y'] > SBND_AV[0]['ymin'])
             & (df['hnl_decay_Y'] < SBND_AV[1]['ymax'])
             & (df['hnl_decay_Z'] > SBND_AV[0]['zmin'])
             & (df['hnl_decay_Z'] < SBND_AV[1]['zmax'])]
    #SIMULATED POT
    simpot = df['pot'].sum()

    #predicted POT for SBND
    totalpot = 10**21

    #Deweight
    df['dew'] = df['fluxw']*df['rayw']*df['decayw'] / simpot * totalpot
    

    return df

def correct_Z(df):
     
    hnlpX = df['hnl_mom_pX']
    hnlpY = df['hnl_mom_pY']
    hnlpZ = df['hnl_mom_pZ']
    hnlE = df['hnl_mom_E']

    hnldposZ = df['hnl_decay_Z']
    hnldposT = df['hnl_decay_t']
    
    hnlcorr = []
    
    for (X, Y, Z, E, z, t) in zip(hnlpX, hnlpY, hnlpZ, hnlE, hnldposZ, hnldposT):
        
        vec = TLorentzVector (X, Y, Z, E)
        vel = vec.Beta() * c_cm_per_ns
    
        tofcorr = z / vel
        hnlcorr.append(tofcorr)
    
    df['hnl_enter_t'] = df['hnl_decay_t'] - hnlcorr

    return df

In [ ]:
def computeAngleP1P2(x1, y1, z1, x2, y2, z2):
    p1Dir = np.array([x1, y1, z1])
    p2Dir = np.array([x2, y2, z2])
    
    #ANGLE W.R.T BEAM LINE
    cos = np.dot(p1Dir, p2Dir) / (np.linalg.norm(p1Dir) * np.linalg.norm(p2Dir))
    th = math.acos(cos)
    th = th * 180 / math.pi
    
    return th

def computeAngle2Beam(x, y , z):
    pDir = np.array([x, y, z])
    
    #Beam
    beamDir = np.array([0, 0, 1])    
    
    #ANGLE W.R.T BEAM LINE
    cos = np.dot(pDir, beamDir) / (np.linalg.norm(pDir) * np.linalg.norm(beamDir))
    th = math.acos(cos)
    th = th * 180 / math.pi
    
    return th

def compute_things_plz(df):
    muX = df['daughter_mom_pX'].loc[abs(df['daughter_pdg'])!=111]
    muY = df['daughter_mom_pY'].loc[abs(df['daughter_pdg'])!=111]
    muZ = df['daughter_mom_pZ'].loc[abs(df['daughter_pdg'])!=111]

    piX = df['daughter_mom_pX'].loc[abs(df['daughter_pdg'])==111]
    piY = df['daughter_mom_pY'].loc[abs(df['daughter_pdg'])==111]
    piZ = df['daughter_mom_pZ'].loc[abs(df['daughter_pdg'])==111]

    hnlX = df['hnl_mom_pX'].xs(0, level='subentry')
    hnlY = df['hnl_mom_pY'].xs(0, level='subentry')
    hnlZ = df['hnl_mom_pZ'].xs(0, level='subentry')

    kaonX = df['kaon_mom_pX'].xs(0, level='subentry')
    kaonY = df['kaon_mom_pY'].xs(0, level='subentry')
    kaonZ = df['kaon_mom_pZ'].xs(0, level='subentry')
    
    openAngle = []
        
    muAngle2HNL = []
    piAngle2HNL = []
    
    muAngle2Beam = []
    piAngle2Beam = []
    HNLAngle2Beam = []
    kAngle2Beam = []

    for (x1, y1, z1, x2, y2, z2, X, Y, Z, kX, kY, kZ) in zip(muX, muY, muZ, piX, piY, piZ, hnlX, hnlY, hnlZ, kaonX, kaonY, kaonZ):
        
        #open angle between mu and pi
        openAngleMuPi = computeAngleP1P2(x1, y1, z1, x2, y2, z2)
        openAngle.append(openAngleMuPi)
        
        #MU ANGLE W.R.T HNL
        openAngleMuHNL = computeAngleP1P2(x1, y1, z1, X, Y, Z)
        muAngle2HNL.append(openAngleMuHNL)
        
        #Pi ANGLE W.R.T HNL
        openAnglePiHNL = computeAngleP1P2(x2, y2, z2, X, Y, Z)
        piAngle2HNL.append(openAnglePiHNL)

        #MU ANGLE W.R.T BEAM LINE
        muAngle2Beam.append(computeAngle2Beam(x1, y1, z1))

        #PI ANGLE W.R.T BEAM LINE
        piAngle2Beam.append(computeAngle2Beam(x2, y2, z2))

        #HNL ANGLE W.R.T BEAM LINE
        HNLAngle2Beam.append(computeAngle2Beam(X, Y, Z))
        
        #Kaon ANGLE W.R.T BEAM LINE
        kAngle2Beam.append(computeAngle2Beam(kX, kY, kZ))
    
    df['angle2beam'] = -9999
    df['angle2HNL'] = -9999
    df['HNLAngle2Beam'] = -9999
    df['KaonAngle2Beam'] = -9999
    
    
    df['angle2beam'].loc[abs(df['daughter_pdg'])!=111] = muAngle2Beam
    df['angle2beam'].loc[abs(df['daughter_pdg'])==111] = piAngle2Beam
    
    df['angle2HNL'].loc[abs(df['daughter_pdg'])!=111] = muAngle2HNL
    df['angle2HNL'].loc[abs(df['daughter_pdg'])==111] = piAngle2HNL

    df['HNLAngle2Beam'].loc[abs(df['daughter_pdg'])!=111] = HNLAngle2Beam
    df['HNLAngle2Beam'].loc[abs(df['daughter_pdg'])==111] = HNLAngle2Beam
    
    df['KaonAngle2Beam'].loc[abs(df['daughter_pdg'])!=111] = kAngle2Beam
    df['KaonAngle2Beam'].loc[abs(df['daughter_pdg'])==111] = kAngle2Beam
    
    return df


In [ ]:
rootpath = [
            '~/HNL_study/ana_file/ana_nupi0_ESP2023/hist_nupi0_gen_m140_UmU_1p00e07.root'
            ,'~/HNL_study/ana_file/ana_nupi0_ESP2023/hist_nupi0_gen_m160_UmU_1p00e07.root'
            ,'~/HNL_study/ana_file/ana_nupi0_ESP2023/hist_nupi0_gen_m180_UmU_1p00e07.root'
            ,'~/HNL_study/ana_file/ana_nupi0_ESP2023/hist_nupi0_gen_m200_UmU_1p00e07.root'
            ,'~/HNL_study/ana_file/ana_nupi0_ESP2023/hist_nupi0_gen_m220_UmU_1p00e07.root'
            ,'~/HNL_study/ana_file/ana_nupi0_ESP2023/hist_nupi0_gen_m240_UmU_1p00e07.root'
            ,'~/HNL_study/ana_file/ana_nuee_ESP2023/hists_prod_BNB_HNL_sbnd_Gen-20230504T225504.root'
            ]
figpath = './plots/'

df140 = read_hnlroot2df(rootpath[0])
df140 = prep_hnldf(df140)
df140 = correct_Z(df140)

df160 = read_hnlroot2df(rootpath[1])
df160 = prep_hnldf(df160)
df160 = correct_Z(df160)

df180 = read_hnlroot2df(rootpath[2])
df180 = prep_hnldf(df180)
df180 = correct_Z(df180)

df200 = read_hnlroot2df(rootpath[3])
df200 = prep_hnldf(df200)
df200 = correct_Z(df200)

df220 = read_hnlroot2df(rootpath[4])
df220 = prep_hnldf(df220)
df220 = correct_Z(df220)

df240 = read_hnlroot2df(rootpath[5])
df240 = prep_hnldf(df240)
df240 = correct_Z(df240)

df30 = read_hnlroot2df(rootpath[6])
df30 = prep_hnldf(df30)
df30 = correct_Z(df30)

df140 = compute_things_plz(df140)
df160 = compute_things_plz(df160)
df180 = compute_things_plz(df180)
df200 = compute_things_plz(df200)
df220 = compute_things_plz(df220)
df240 = compute_things_plz(df240)

In [ ]:
m_list = [140, 160, 180, 200, 220, 240]

df_list = [df140, df160, df180, df200, df220, df240]

col_list = {
    140: col_dict['Lavender']
    ,160: col_dict['Purple']
    ,180: col_dict['Aqua']
    ,200: col_dict['Spearmint']
    ,220: col_dict['Peach']
    ,240: col_dict['Coral']
    
}


<h1>Energy Spectrum</h1>

In [ ]:
fig, ax1 = plt.subplots(1,1, figsize=(8,6))

xmin = 0
xmax = 8
xnbin = 40
xlimmin = xmin
xlimmax = xmax

title = r"Normalised to $|U_{\mu4}|^2 = 1 \times 10^{-7}$ and $1 \times 10^{21}$ POT"
xtitle = r"E$_{N}$ [GeV]"
binsize = (xmax-xmin)/xnbin
ytitle = f"Entries / {round(binsize*100)} MeV"

for m, df in zip(reversed(m_list), reversed(df_list)):
    
    #----------------------------------------------------------------------
    pltdf = df['hnl_mom_E'].loc[abs(df['daughter_pdg'])==111]
    deweight = df['dew'].loc[abs(df['daughter_pdg'])==111]

    hnl_label = str(m) + ' MeV HNL'
    plot_1dhist(
                            pltdf, ax1,
                            xmin, xmax, xnbin,xlimmin, xlimmax,
                            weights = deweight, ifnorm = False,
                            title=title,xtitle = xtitle, ytitle = ytitle,
                            iflabelbox = True, label = hnl_label, histtype = 'step', 
                            linecolor = col_list[m], color = 'white',linewidth = 2,
                            ifysci = True,
                        )
    
#-------------------------------------------------------------------
handles, labels = plt.gca().get_legend_handles_labels()
handles = handles[::-1]
labels = labels[::-1]

ax1.legend(handles, labels, loc ='best', fontsize = 16, ncol = 2)
#-------------------------------------------------------------------
ax1.tick_params(bottom = True, top = False, left = True, right = False) 

ax1.tick_params(axis = 'x', labelsize = 16, direction = 'in')
ax1.tick_params(axis = 'y', labelsize = 16, direction = 'out')
ax1.xaxis.set_major_locator(MaxNLocator(5))
ax1.yaxis.get_major_locator().set_params(numticks=99)
ax1.yaxis.get_minor_locator().set_params(numticks=99)#, subs=[.2, .4, .6, .8])
plt.ylim(10**-5, 10**5)

#----------------------------------------------------------------------
fig.tight_layout()
pltname = "HNL_TrueEnergy_Spectrum"
plt.savefig(figpath+pltname+'.png', dpi = 100)
plt.show()

In [ ]:
fig, ax1 = plt.subplots(1,1, figsize=(8,6))

xmin = 0
xmax = 8
xnbin = 40
xlimmin = xmin
xlimmax = xmax

title = r""
xtitle = r"E$_{\pi^0}$ [GeV]"
binsize = (xmax-xmin)/xnbin
ytitle = f"Entries / {round(binsize*100)} MeV / a.u."

for m, df in zip(m_list, df_list):
#for m, df in zip(reversed(m_list), reversed(df_list)):
    
    #----------------------------------------------------------------------
    pltdf = df['daughter_mom_E'].loc[abs(df['daughter_pdg'])==111]
    deweight = df['dew'].loc[abs(df['daughter_pdg'])==111]

    hnl_label = str(m) + ' MeV HNL'
    plot_1dhist(
                            pltdf, ax1,
                            xmin, xmax, xnbin,xlimmin, xlimmax,
                            weights = None, ifnorm = True,
                            xtitle = xtitle, ytitle = ytitle,
                            iflabelbox = True, label = hnl_label, histtype = 'step',
                            linecolor = col_list[m], color = 'white', linewidth = 2,
                            ifysci = True,
                        )

#-------------------------------------------------------------------
ax1.tick_params(bottom = True, top = False, left = True, right = False) 

ax1.tick_params(axis = 'x', labelsize = 16, direction = 'in')
ax1.tick_params(axis = 'y', labelsize = 16, direction = 'out')
ax1.xaxis.set_major_locator(MaxNLocator(5))
ax1.yaxis.get_major_locator().set_params(numticks=99)
ax1.yaxis.get_minor_locator().set_params(numticks=99)#, subs=[.2, .4, .6, .8])
plt.ylim(10**-4, 10)
ax1.legend(loc ='best', fontsize = 16, ncol = 2)
#----------------------------------------------------------------------
fig.tight_layout()
pltname = "pi0_TrueEnergy_Spectrum"
plt.savefig(figpath+pltname+'.png', dpi = 100)
plt.show()

<h1>Angle To The Beam</h1>

In [ ]:
fig, ax1 = plt.subplots(1,1, figsize=(8,6))

xmin = 0
xmax = 180
xnbin = 180
xlimmin = xmin
xlimmax = xmax

title = ""
xtitle = r"$\pi^0$ Theta [degrees]"
binsize = (xmax-xmin)/xnbin
ytitle = "Entries [a.u.]"

for m, df in zip(reversed(m_list), reversed(df_list)):
#for m, df in zip(m_list, df_list):    
    #----------------------------------------------------------------------
    pltdf = df['KaonAngle2Beam'].loc[(abs(df['daughter_pdg'])==111)]
    deweight = df['dew'].loc[(abs(df['daughter_pdg'])==111) ]

    hnl_label = str(m) + ' MeV'
    plot_1dhist(
                            pltdf, ax1,
                            xmin, xmax, xnbin,xlimmin, xlimmax,
                            weights = None, ifnorm = True,
                            xtitle = xtitle, ytitle = ytitle,
                            iflabelbox = True, label = hnl_label, histtype = 'step',
                            linecolor = col_list[m], color = 'white', linewidth = 2,
                            ifysci = False,
                        )

#-------------------------------------------------------------------
ax1.tick_params(bottom = True, top = False, left = True, right = False)
ax1.tick_params(axis = 'x', labelsize = 16, direction = 'in')
ax1.tick_params(axis = 'y', labelsize = 16, direction = 'out')
ax1.xaxis.set_major_locator(MaxNLocator(5))
ax1.yaxis.get_major_locator().set_params(numticks=99)
ax1.yaxis.get_minor_locator().set_params(numticks=99)#, subs=[.2, .4, .6, .8])
plt.ylim(10**-4, 1)
ax1.legend(loc ='best', fontsize = 16, ncol = 2)
#----------------------------------------------------------------------
fig.tight_layout()
pltname = "kaon_angle2Beam"
#plt.savefig(figpath+pltname+'.png', dpi = 100)
plt.show()

In [ ]:
fig, ax1 = plt.subplots(1,1, figsize=(8,6))

xmin = 0
xmax = 5
xnbin = 5
xlimmin = xmin
xlimmax = xmax

title = ""
xtitle = r"HNL Theta [degrees]"
binsize = (xmax-xmin)/xnbin
ytitle = "Entries [a.u.]"

for m, df in zip(reversed(m_list), reversed(df_list)):

    #----------------------------------------------------------------------
    pltdf = df['HNLAngle2Beam'].loc[abs(df['daughter_pdg'])!=111]
    deweight = df['dew'].loc[abs(df['daughter_pdg'])!=111]

    hnl_label = str(m) + ' MeV'
    plot_1dhist(
                            pltdf, ax1,
                            xmin, xmax, xnbin,xlimmin, xlimmax,
                            weights = deweight, ifnorm = True,
                            xtitle = xtitle, ytitle = ytitle,
                            iflabelbox = True, label = hnl_label, histtype = 'step', 
                            linecolor = col_list[m], color = 'white', linewidth = 2,
                            ifysci = False,
                        )
        #----------------------------------------------------------------------

#-------------------------------------------------------------------
ax1.tick_params(bottom = True, top = False, left = True, right = False) 
ax1.xaxis.set_major_locator(MaxNLocator(5))
#ax1.yaxis.get_major_locator().set_params(numticks=99)
#ax1.yaxis.get_minor_locator().set_params(numticks=99)#, subs=[.2, .4, .6, .8])
#plt.ylim(10**-4, 10**5)
ax1.legend(loc ='best', fontsize = 16, ncol = 1)
#----------------------------------------------------------------------
fig.tight_layout()
pltname = "HNL_angle2Beam"
#plt.savefig(figpath+pltname+'.png', dpi = 100)
plt.show()

In [ ]:
fig, ax1 = plt.subplots(1,1, figsize=(8,6))

xmin = 0
xmax = 65
xnbin = 65
xlimmin = xmin
xlimmax = xmax

title = ""
xtitle = r"$\theta_{\pi^0}$ [GeV]"
binsize = (xmax-xmin)/xnbin
ytitle = f"Entries / {round(binsize)} degrees / a.u."

for m, df in zip(reversed(m_list), reversed(df_list)):
#for m, df in zip(m_list, df_list):    
    #----------------------------------------------------------------------
    pltdf = df['angle2beam'].loc[(abs(df['daughter_pdg'])==111)]
    deweight = df['dew'].loc[(abs(df['daughter_pdg'])==111) ]

    hnl_label = str(m) + ' MeV HNL'
    plot_1dhist(
                            pltdf, ax1,
                            xmin, xmax, xnbin,xlimmin, xlimmax,
                            weights = None, ifnorm = True,
                            xtitle = xtitle, ytitle = ytitle,
                            iflabelbox = True, label = hnl_label, histtype = 'step',
                            linecolor = col_list[m], color = 'white', linewidth = 2,
                            ifysci = True,
                        )
#-------------------------------------------------------------------
handles, labels = plt.gca().get_legend_handles_labels()
handles = handles[::-1]
labels = labels[::-1]

ax1.legend(handles, labels, loc ='best', fontsize = 16, ncol = 2)
#-------------------------------------------------------------------
ax1.tick_params(bottom = True, top = False, left = True, right = False)
ax1.tick_params(axis = 'x', labelsize = 16, direction = 'in')
ax1.tick_params(axis = 'y', labelsize = 16, direction = 'out')
ax1.xaxis.set_major_locator(MaxNLocator(5))
ax1.yaxis.get_major_locator().set_params(numticks=99)
ax1.yaxis.get_minor_locator().set_params(numticks=99)#, subs=[.2, .4, .6, .8])
plt.ylim(10**-4, 1)
#----------------------------------------------------------------------
fig.tight_layout()
pltname = "pi0_angle2Beam"
plt.savefig(figpath+pltname+'.png', dpi = 100)
plt.show()

<h1>Timing</h1>

In [ ]:
fig, ax1 = plt.subplots(1,1, figsize=(18,6))

xmin = 340
xmax = xmin + 1600
xnbin = 1600
xlimmin = xmin
xlimmax = xmax

xtitle = "Arrival Time [ns]"
title = ""

binsize = (xmax-xmin)/xnbin
ytitle = f"Entries / {round(binsize)} ns / a.u."
#----------------------------------------------------------------------
pltdf = dfNU['nu_enterT']

_ , _, _ = plot_1dhist(
                        pltdf,
                        ax1,
                        xmin, xmax, xnbin,
                        xlimmin, xlimmax,
                        title = title, xtitle = xtitle, ytitle = ytitle, histtype = 'step',
                        linestyle='dashed',linewidth = 1.5,
                        iflabelbox = True, label = "BNB Neutrinos", ifnorm = True,
                        linecolor = col_dict['PastelGreen']
                    )
#---------------------------------------------------------------------
pltdf = df240['hnl_enter_t'].loc[abs(df240['daughter_pdg'])!=111]

_ , _, _ = plot_1dhist(
                        pltdf,
                        ax1,
                        xmin, xmax, xnbin,
                        xlimmin, xlimmax,
                        linestyle = 'solid', linewidth = 2,
                        title = title, xtitle = xtitle, ytitle = ytitle, histtype = 'step',
                        iflabelbox = True, label = r"240 MeV HNL", ifnorm = True,
                        linecolor = col_list[240]
                    )

#----------------------------------------------------------------------
plt.ylim(0, 0.006)
ax1.tick_params(axis = 'x', labelsize = 16, direction = 'in')
ax1.tick_params(axis = 'y', labelsize = 16, direction = 'out')
plt.legend(loc ='upper left', fontsize =16, ncol = 2)
ax1.tick_params(bottom = True, top = False, left = True, right = False) 
ax1.xaxis.set_major_locator(MaxNLocator(5))
#----------------------------------------------------------------------

fig.tight_layout()
pltname = "full_beam"
plt.savefig(figpath+pltname+'.png', dpi = 100)
plt.show()

In [ ]:
width = 19

dfNU['mod_t'] = dfNU['nu_enterT'] % width

df140['mod_t'] = df140['hnl_enter_t'] % width
df160['mod_t'] = df160['hnl_enter_t'] % width
df180['mod_t'] = df180['hnl_enter_t'] % width
df200['mod_t'] = df200['hnl_enter_t'] % width
df220['mod_t'] = df220['hnl_enter_t'] % width
df240['mod_t'] = df240['hnl_enter_t'] % width

In [ ]:
fig, ax1 = plt.subplots(1,1, figsize=(8,6))

xmin = 0
xmax = 30
xnbin = 30
xlimmin = xmin
xlimmax = xmax
xtitle = "Arrival Time % 18.936 [ns]"
title = ""

binsize = (xmax-xmin)/xnbin
ytitle = f"Entries / {round(binsize)} ns / a.u."
#----------------------------------------------------------------------
pltdf = dfNU['mod_t']

_ , _, _ = plot_1dhist(
                        pltdf,
                        ax1,
                        xmin, xmax, xnbin,
                        xlimmin, xlimmax,
                        title = title, xtitle = xtitle, ytitle = ytitle, histtype = 'step',
                        linestyle='dashed',linewidth = 1.5,
                        iflabelbox = True, label = "BNB Neutrinos", ifnorm = True,
                        linecolor = col_dict['PastelGreen']
                    )

#for m, df in zip(reversed(m_list), reversed(df_list)):
for m, df in zip(m_list, df_list):  
    #----------------------------------------------------------------------
    pltdf = df['mod_t'].loc[abs(df['daughter_pdg'])!=111]
    deweight = df['dew'].loc[abs(df['daughter_pdg'])!=111]

    hnl_label = str(m) + ' MeV HNL'
    plot_1dhist(
                            pltdf, ax1,
                            xmin, xmax, xnbin,xlimmin, xlimmax,
                            weights = None, ifnorm = True,
                            xtitle = xtitle, ytitle = ytitle, histtype = 'step',
                            iflabelbox = True, label = hnl_label, 
                            linecolor = col_list[m], linewidth = 2,
                            ifysci = False,
                        )
#----------------------------------------------------------------------
#plt.ylim(0, 0.15)
#plt.xlim(367, 380)
ax1.tick_params(axis = 'x', labelsize = 16, direction = 'in')
ax1.tick_params(axis = 'y', labelsize = 16, direction = 'out')
plt.legend(loc ='upper right', fontsize =16)
ax1.tick_params(bottom = True, top = False, left = True, right = False) 
ax1.xaxis.set_major_locator(MaxNLocator(5))
#----------------------------------------------------------------------

fig.tight_layout()
pltname = "beam_modulus"
plt.savefig(figpath+pltname+'.png', dpi = 100)
plt.show()

<h1>Sanity Check</h1>

In [ ]:
def hnl_momentum(kaon_mass, lep_mass, hnl_mass):
    if (kaon_mass - lep_mass < hnl_mass):
        return -1.

    return np.sqrt(kaon_mass * kaon_mass * kaon_mass * kaon_mass 
    -2 * kaon_mass * kaon_mass * lep_mass * lep_mass 
    -2 * kaon_mass * kaon_mass * hnl_mass * hnl_mass 
       + lep_mass * lep_mass * lep_mass * lep_mass 
       + hnl_mass * hnl_mass * hnl_mass * hnl_mass 
    -2 * lep_mass * lep_mass * hnl_mass * hnl_mass) / ( 2 * kaon_mass )

    
def RandomUnitVector():
    #In order to pick a random point on a sphere -- pick a random value of _costh_, __not__ theta
    #b.c. d\Omega = d\phi dcos\theta, i.e. d\Omega != d\phi d\theta
    costheta = random.uniform(-1, 1)
    sintheta = np.sqrt(1. - costheta * costheta)
    phi = random.uniform(0, 2*math.pi)
    
    return [sintheta * math.cos(phi), sintheta * math.sin(phi), costheta]

In [ ]:
kaon_mass = 0.493677
muon_mass = 0.105658

k_px = -0.025774
k_py = 0.457946
k_pz = 0.158222
k_e = 0.692192
k_THETA = 70.967767

h_px = 0.001792
h_py = -0.003143
h_pz = 0.148389
h_e = 0.218248
h_THETA = 1.396618

kaon_mom = Momentum4(k_e, k_px, k_py, k_pz)
kaon_mom

In [ ]:
hnl_mass = 0.240

# get the momentum direction in the kaon parent rest frame
p = hnl_momentum(kaon_mass, muon_mass, hnl_mass)
e = np.sqrt(p*p + hnl_mass * hnl_mass)

v = RandomUnitVector()
print(v)

hnl_mom = Momentum4(e, p*h_px, p*h_py, p*h_pz)
hnl_mom

hnl_mom_boost = hnl_mom.boost_particle(kaon_mom)
hnl_mom_boost

In [ ]:
print(df240['KaonAngle2Beam'][(df240['KaonAngle2Beam'] < 180) & (df240['KaonAngle2Beam'] > 150)])
print(df240['KaonAngle2Beam'][(df240['KaonAngle2Beam'] < 150) & (df240['KaonAngle2Beam'] > 120)])
print(df240['KaonAngle2Beam'][(df240['KaonAngle2Beam'] < 120) & (df240['KaonAngle2Beam'] > 90)])
print(df240['KaonAngle2Beam'][(df240['KaonAngle2Beam'] < 90) & (df240['KaonAngle2Beam'] > 60)])
print(df240['KaonAngle2Beam'][(df240['KaonAngle2Beam'] < 60) & (df240['KaonAngle2Beam'] > 30)])
print(df240['KaonAngle2Beam'][(df240['KaonAngle2Beam'] < 30) & (df240['KaonAngle2Beam'] > 0)])

In [ ]:
angle180 = df240[np.isclose(df['KaonAngle2Beam'], 173.301978)]
angle150 = df240[np.isclose(df['KaonAngle2Beam'], 152.576123)]
angle120 = df240[np.isclose(df['KaonAngle2Beam'], 122.248125)]
angle90 = df240[np.isclose(df['KaonAngle2Beam'], 94.088303)]
angle60 = df240[np.isclose(df['KaonAngle2Beam'], 63.487781)]
angle30 = df240[np.isclose(df['KaonAngle2Beam'], 34.113190)]
angle0 = df240[np.isclose(df['KaonAngle2Beam'], 9.925133)]

In [ ]:
kE = [angle180['kaon_mom_E'].xs(0, level='subentry').iloc[0]
       , angle150['kaon_mom_E'].xs(0, level='subentry').iloc[0]
       , angle120['kaon_mom_E'].xs(0, level='subentry').iloc[0]
       , angle90['kaon_mom_E'].xs(0, level='subentry').iloc[0]
       , angle60['kaon_mom_E'].xs(0, level='subentry').iloc[0]
       , angle30['kaon_mom_E'].xs(0, level='subentry').iloc[0]
       , angle0['kaon_mom_E'].xs(0, level='subentry').iloc[0]
      ]
print(kE)

In [ ]:
kth = [angle180['KaonAngle2Beam'].xs(0, level='subentry').iloc[0]
       , angle150['KaonAngle2Beam'].xs(0, level='subentry').iloc[0]
       , angle120['KaonAngle2Beam'].xs(0, level='subentry').iloc[0]
       , angle90['KaonAngle2Beam'].xs(0, level='subentry').iloc[0]
       , angle60['KaonAngle2Beam'].xs(0, level='subentry').iloc[0]
       , angle30['KaonAngle2Beam'].xs(0, level='subentry').iloc[0]
       , angle0['KaonAngle2Beam'].xs(0, level='subentry').iloc[0]
      ]

print(kth)

hth = [angle180['HNLAngle2Beam'].xs(0, level='subentry').iloc[0]
       , angle150['HNLAngle2Beam'].xs(0, level='subentry').iloc[0]
       , angle120['HNLAngle2Beam'].xs(0, level='subentry').iloc[0]
       , angle90['HNLAngle2Beam'].xs(0, level='subentry').iloc[0]
       , angle60['HNLAngle2Beam'].xs(0, level='subentry').iloc[0]
       , angle30['HNLAngle2Beam'].xs(0, level='subentry').iloc[0]
       , angle0['HNLAngle2Beam'].xs(0, level='subentry').iloc[0]
      ]

print(hth)

kth = np.deg2rad(kth)
hth = np.deg2rad(hth)

In [ ]:
from matplotlib.pyplot import cm
polar_color = cm.plasma(np.linspace(0, 1, len(kth)))

polar_color = [
    col_dict['Lavender']
    ,col_dict['Purple']
    ,col_dict['Teal']
    ,col_dict['Aqua']
    ,col_dict['Spearmint']
    ,col_dict['Peach']
    ,col_dict['Coral']
]

In [ ]:
# Creating a new figure and setting up the resolution
fig = plt.figure(figsize =[8,6])

# Change the coordinate system from scaler to polar
ax = fig.add_subplot(111, projection='polar')
ax.set_thetamax(180)

# Define radius
r=10

# plotting the polar coordinates on the system
for c, k, h in zip(polar_color, kth, hth):
    plt.vlines(k, 0, r, color = c, ls = '--', lw = 2)

#remove ytick
ax.set_yticklabels([])

# Setting the axis limit
ax.set_ylim(0,10)

ax.tick_params(axis = 'x', labelsize = 16, direction = 'out')
# Displaying the plot
#----------------------------------------------------------------------

fig.tight_layout()
pltname = "kaon_angle"
plt.savefig(figpath+pltname+'.png', dpi = 100)
plt.show()

In [ ]:
# Creating a new figure and setting up the resolution
fig = plt.figure(figsize =[8,6])

# Change the coordinate system from scaler to polar
ax = fig.add_subplot(111, projection='polar')
ax.set_thetamax(180)

# Define radius
r=10

# plotting the polar coordinates on the system
for c, k, h in zip(polar_color, kth, hth):
    plt.vlines(h, 0, r, color = c, ls = '-', lw = 2)

#remove ytick
ax.set_yticklabels([])

# Setting the axis limit
ax.set_ylim(0,10)

ax.tick_params(axis = 'x', labelsize = 16, direction = 'out')
# Displaying the plot
#----------------------------------------------------------------------

fig.tight_layout()
pltname = "hnl_angle"
plt.savefig(figpath+pltname+'.png', dpi = 100)
plt.show()